In [ ]:
!nvidia-smi

Tue Oct  8 20:11:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pwd # /content/sample_data

/content


In [ ]:
!pip install -q supervision ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.5/882.5 kB 33.4 MB/s eta 0:00:00


In [ ]:
import cv2

import numpy as np
import supervision as sv

from tqdm import tqdm
from ultralytics import YOLO
from supervision.assets import VideoAssets, download_assets
from collections import defaultdict, deque

In [ ]:
download_assets(VideoAssets.VEHICLES)

  0%|          | 0/35345757 [00:00<?, ?it/s]

'vehicles.mp4'

In [ ]:
SOURCE_VIDEO_PATH = "vehicles.mp4"
TARGET_VIDEO_PATH = "vehicles-result.mp4"
CONFIDENCE_THRESHOLD = 0.3
IOU_THRESHOLD = 0.5
MODEL_NAME = "yolov8x.pt"
MODEL_RESOLUTION = 1280

In [ ]:
SOURCE = np.array([
    [1252, 787],
    [2298, 803],
    [5039, 2159],
    [-550, 2159]
])

TARGET_WIDTH = 25
TARGET_HEIGHT = 250

TARGET = np.array([
    [0, 0],
    [TARGET_WIDTH - 1, 0],
    [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
    [0, TARGET_HEIGHT - 1],
])

In [ ]:
import cv2
import os

import numpy as np
import supervision as sv

from tqdm import tqdm
from ultralytics import YOLO
from supervision.assets import VideoAssets, download_assets
from collections import defaultdict, deque

# Verify if the file exists and print its details
SOURCE_VIDEO_PATH = "vehicles.mp4"
if os.path.exists(SOURCE_VIDEO_PATH):
    print(f"File exists at: {SOURCE_VIDEO_PATH}")
    print(f"File size: {os.path.getsize(SOURCE_VIDEO_PATH)} bytes")
else:
    print(f"File not found at: {SOURCE_VIDEO_PATH}")
    # Optionally, handle the missing file:
    raise FileNotFoundError(f"Video file not found at {SOURCE_VIDEO_PATH}")

# Try opening the video with cv2.VideoCapture
try:
    cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
    if not cap.isOpened():
        raise IOError  # Raise an exception if it fails
    print("Video opened successfully.")
    cap.release()  # Release the video capture object
except IOError:
    print(f"Error: Could not open video at {SOURCE_VIDEO_PATH}")
    print("Possible causes:")
    print("- Incorrect file path")
    print("- File permissions")
    print("- Unsupported video format/codec")
    # Consider alternative solutions or error handling

# Custom video frame generator
def get_video_frames_generator(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # End of video
        yield frame
    cap.release()

# Use the custom generator
frame_generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
frame_iterator = iter(frame_generator)

try:
    frame = next(frame_iterator)  # Try to get the first frame
    print("First frame obtained successfully.")
    print(f"Frame shape: {frame.shape}")  # Debug frame shape
except StopIteration:
    print("No frames available from the video.")


File exists at: vehicles.mp4
File size: 35345757 bytes
Video opened successfully.
First frame obtained successfully.
Frame shape: (2160, 3840, 3)


In [ ]:
class ViewTransformer:

    def __init__(self, source: np.ndarray, target: np.ndarray) -> None:
        source = source.astype(np.float32)
        target = target.astype(np.float32)
        self.m = cv2.getPerspectiveTransform(source, target)

    def transform_points(self, points: np.ndarray) -> np.ndarray:
        if points.size == 0:
            return points

        reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
        transformed_points = cv2.perspectiveTransform(reshaped_points, self.m)
        return transformed_points.reshape(-1, 2)

In [ ]:
view_transformer = ViewTransformer(source=SOURCE, target=TARGET)

In [ ]:
!pip install --upgrade supervision
# make sure to restart the runtime to use the updated package

import supervision as sv
model = YOLO(MODEL_NAME)

video_info = sv.VideoInfo.from_video_path(video_path=SOURCE_VIDEO_PATH)
frame_generator = sv.get_video_frames_generator(source_path=SOURCE_VIDEO_PATH)

# tracer initiation
byte_track = sv.ByteTrack(
    frame_rate=video_info.fps, track_activation_threshold=CONFIDENCE_THRESHOLD
)

# annotators configuration
thickness = sv.draw.utils.calculate_optimal_line_thickness( # Changed function name to calculate_optimal_line_thickness
    resolution_wh=video_info.resolution_wh
)
text_scale = sv.draw.utils.calculate_optimal_text_scale( # Use calculate_optimal_text_scale
    resolution_wh=video_info.resolution_wh
)
bounding_box_annotator = sv.BoundingBoxAnnotator(
    thickness=thickness
)
label_annotator = sv.LabelAnnotator(
    text_scale=text_scale,
    text_thickness=thickness,
    text_position=sv.Position.BOTTOM_CENTER
)
trace_annotator = sv.TraceAnnotator(
    thickness=thickness,
    trace_length=video_info.fps * 2,
    position=sv.Position.BOTTOM_CENTER
)

polygon_zone = sv.PolygonZone(
    polygon=SOURCE,
    #frame_resolution_wh=video_info.resolution_wh
)

coordinates = defaultdict(lambda: deque(maxlen=video_info.fps))

# open target video
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:

    # loop over source video frame
    for frame in tqdm(frame_generator, total=video_info.total_frames):

        result = model(frame, imgsz=MODEL_RESOLUTION, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(result)

        # filter out detections by class and confidence
        detections = detections[detections.confidence > CONFIDENCE_THRESHOLD]
        detections = detections[detections.class_id != 0]

        # filter out detections outside the zone
        detections = detections[polygon_zone.trigger(detections)]

        # refine detections using non-max suppression
        detections = detections.with_nms(IOU_THRESHOLD)

        # pass detection through the tracker
        detections = byte_track.update_with_detections(detections=detections)

        points = detections.get_anchors_coordinates(
            anchor=sv.Position.BOTTOM_CENTER
        )

        # calculate the detections position inside the target RoI
        points = view_transformer.transform_points(points=points).astype(int)

        # store detections position
        for tracker_id, [_, y] in zip(detections.tracker_id, points):
            coordinates[tracker_id].append(y)

        # format labels
        labels = []

        for tracker_id in detections.tracker_id:
            if len(coordinates[tracker_id]) < video_info.fps / 2:
                labels.append(f"#{tracker_id}")
            else:
                # calculate speed
                coordinate_start = coordinates[tracker_id][-1]
                coordinate_end = coordinates[tracker_id][0]
                distance = abs(coordinate_start - coordinate_end)
                time = len(coordinates[tracker_id]) / video_info.fps
                speed = distance / time * 3.6
                labels.append(f"#{tracker_id} {int(speed)} km/h")

        # annotate frame
        annotated_frame = frame.copy()
        annotated_frame = trace_annotator.annotate(
            scene=annotated_frame, detections=detections
        )
        annotated_frame = bounding_box_annotator.annotate(
            scene=annotated_frame, detections=detections
        )

        annotated_frame = label_annotator.annotate(
            scene=annotated_frame, detections=detections, labels=labels
        )

        # add frame
        # add frame to target video
        sink.write_frame(annotated_frame)

SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
100%|██████████| 538/538 [02:08<00:00,  4.18it/s]


In [ ]:
import ffmpeg

def convert_mp4_to_mkv(input_file, output_file):
    try:
        ffmpeg.input(input_file).output(output_file).run()
        print(f"Successfully converted {input_file} to {output_file}")
    except ffmpeg.Error as e:
        print(f"Error converting {input_file} to {output_file}: {e.stderr}")

# Example usage:
input_file = '/path/to/input_video.mp4'  # Replace with your input MP4 file path
output_file = 'try 1.mkv'  # Replace with your desired output MKV file path

convert_mp4_to_mkv(input_file, output_file)


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the pre-trained YOLOv8 model
model = YOLO('yolov8n.pt')

# Define car color labels
CAR_COLORS = ['black', 'green', 'yellow', 'red', 'white', 'blue']

# Load video file
video_path = 'try 1.mkv'
#video_path = './000.mp4'
cap = cv2.VideoCapture(video_path)

# Get video width, height, and frame rate
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output video save settings
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('./try 02.mp4', fourcc, fps, (width, height))


def detect_color(image):
    # Convert image from BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define color ranges (set based on empirical values)
    color_ranges = {
        'black': [(0, 0, 0), (180, 255, 50)],
        'green': [(30, 40, 40), (90, 255, 255)],
        'yellow': [(20, 40, 40), (40, 255, 255)],
        'red': [(0, 70, 50), (10, 255, 255)],
        'white': [(0, 0, 220), (180, 30, 255)],
        'blue': [(90, 50, 50), (150, 255, 255)]  # including light and normal blue
    }

    # Calculate the number of pixels for each color
    max_pixels = 0
    detected_color = "black"
    for color, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        count = cv2.countNonZero(mask)
        if color == 'red':
            # Apply additional condition to differentiate dark red from black
            if count > max_pixels and np.mean(hsv[:, :, 2]) > 50:  # Ensure brightness is above a threshold
                max_pixels = count
                detected_color = color
        elif count > max_pixels:
            max_pixels = count
            detected_color = color

    return detected_color


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Use YOLOv8 to detect cars
    results = model(frame)
    for result in results:
        for box in result.boxes:
            # Get bounding box coordinates and confidence
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0]
            cls = box.cls[0]

            # Detect all vehicle types (e.g., car, truck, bus, motorcycle)
            if cls in [2, 3, 5, 7]:  # 2: car, 3: motorcycle, 5: bus, 7: truck (using COCO dataset class index)
                car_image = frame[y1:y2, x1:x2]

                # Detect car color
                car_color = detect_color(car_image)

                # Draw bounding box and label
                label = f'Color: {car_color}'
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Write to output video
    out.write(frame)

    # Display video frame
    cv2.imshow('Car Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
import ffmpeg

def convert_mp4_to_mkv(input_file, output_file):
    try:
        ffmpeg.input(input_file).output(output_file).run()
        print(f"Successfully converted {input_file} to {output_file}")
    except ffmpeg.Error as e:
        print(f"Error converting {input_file} to {output_file}: {e.stderr}")

# Example usage:
input_file = '/output_video.mp4'  # Replace with your input MP4 file path
output_file = 'try 02.mkv'  # Replace with your desired output MKV file path

convert_mp4_to_mkv(input_file, output_file)

In [ ]:
import cv2
from ultralytics import YOLO
from collections import deque

# Load YOLO model
model = YOLO('./yolov8x.pt')

# Video path
#video_path = '../video/try 1.mkv'
video_path = 'try 02.mkv'
cap = cv2.VideoCapture(video_path)

# Get video's height, width, and fps
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output path
output_path = "./try 03.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Detection label
vehicle_classes = ['car', 'truck']

# Record every car's last center
tracked_cars = {}
current_frame_center = {}
previous_frame_center = {}

# line position
#line1_y = 550
#line2_y = 450
line1_y = 1500
line2_y = 1400

car_in = 0
truck_in = 0
car_out = 0
truck_out = 0

n = 1

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Complete")
        break

    # Use model to predict
    results = model(frame)

    # Draw the line
    color_red = (0, 0, 255)
    color_blue = (255, 0, 0)
    thickness = 2

    cv2.line(frame, (0, line1_y), (frame_width, line1_y), color_red, thickness)
    cv2.line(frame, (0, line2_y), (frame_width, line2_y), color_blue, thickness)

    # Draw the frame and center point
    for result in results[0].boxes:
        # class id
        class_id = int(result.cls)
        # confidence
        confidence = result.conf.item()
        # class_name
        class_name = model.names[class_id]

        if n == 1:
            if confidence > 0.3 and class_name in vehicle_classes:
                x1, y1, x2, y2 = map(int, result.xyxy[0].cpu().numpy())

                # center point
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)

                # Draw the boundary
                label = f"{class_name} {confidence:.2f}"
                color = (0, 0, 0)

                if class_name == "truck":
                    color = (255, 0, 255)
                else:
                    color = (0, 255, 0)

                #cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

                # Draw the center point
                cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)

                # Generate a unique ID
                vehicle_id = f"{class_name}_{center_x}"
                tracked_cars[vehicle_id] = {center_x, center_y}

                n += 1
        else:
            if confidence > 0.3 and class_name in vehicle_classes:
                # Get the boundary position
                x1, y1, x2, y2 = map(int, result.xyxy[0].cpu().numpy())

                # Calculate the center point
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)
                #print("c1", center_x, center_y)

                previous_center_x, previous_center_y = tracked_cars[vehicle_id]

                label = f"{class_name} {confidence:.2f}"
                color = (0, 0, 0)

                if class_name == "truck":
                    color = (255, 0, 255)
                else:
                    color = (0, 255, 0)

                if vehicle_id in tracked_cars:
                    arrow_start = (center_x, center_y)
                    arrow_length = 20
                    prev_center_x, prev_center_y = tracked_cars[vehicle_id]
                    #print("c2", (prev_center_x, prev_center_y))

                    # Determine the direction
                    if center_x > prev_center_x and center_y < prev_center_y:  # Moving forward
                        direction = "Forward"
                        arrow_end = (center_x + arrow_length, center_y - arrow_length)
                    elif center_x < prev_center_x and center_y > prev_center_y:  # Moving backward
                        direction = "Backward"
                        arrow_end = (center_x - arrow_length, center_y + arrow_length)
                    else:
                        direction = "Stationary"
                        arrow_end = arrow_start

                    # line1_y = 550
                    # line2_y = 450
                    if center_y - line1_y >= -10 and center_y in range(line2_y, line1_y):
                        if class_name == "truck":
                            truck_in += 1
                        elif class_name == "car":
                            car_in += 1
                    elif center_y - line2_y <= 5 and center_y in range(line2_y, line1_y):
                        if class_name == "truck":
                            truck_out += 1
                        elif class_name == "car":
                            car_out += 1

                    #cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    # Draw the center point
                    cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)
                    # Draw arrow
                    cv2.arrowedLine(frame, arrow_start, arrow_end, color, 2)
                    # Draw the direction
                    cv2.putText(frame, direction, (center_x, center_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                                (255, 255, 255), 2)

                    cv2.putText(frame, f"car_in: {car_in}", (frame_width - 800, 60), cv2.FONT_HERSHEY_SIMPLEX, 3,
                                (255, 255, 255), 2)
                    cv2.putText(frame, f"truck_in: {truck_in}", (frame_width - 800, 140), cv2.FONT_HERSHEY_SIMPLEX, 3,
                                (255, 255, 255), 2)
                    cv2.putText(frame, f"car_out: {car_out}", (frame_width - 800, 220), cv2.FONT_HERSHEY_SIMPLEX, 3,
                                (255, 255, 255), 2)
                    cv2.putText(frame, f"truck_out: {truck_out}", (frame_width - 800, 300), cv2.FONT_HERSHEY_SIMPLEX, 3,
                                (255, 255, 255), 2)

                    # Update tracked_cars with the current position
                tracked_cars[vehicle_id] = (center_x, center_y)

    out.write(frame)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()